In [ ]:
# This has to go in its own cell or it screws up the defaults we'll set later
%matplotlib inline

In [ ]:
import numpy as np
from scrapbook import plot
from scrapbook.plot import plt as pyplot

In [ ]:
import musictoys
import musictoys.audiofile
import musictoys.analysis
import musictoys.spectral

In [ ]:
def spacefillpath(height, width, dtype=np.int):
    matrix = np.zeros((height, width), dtype=dtype)
    y, x = 1, 1
    while x < width and y < height:
        nextx = min(x*2, width)
        if nextx > x:
            more = nextx - x
            current = matrix[0:y, 0:more]
            matrix[0:y, x:nextx] = current + (y*more)
            x = nextx
        nexty = min(y*2, height)
        if nexty > y:
            more = nexty - y
            current = matrix[0:more, 0:x]
            matrix[y:nexty, 0:x] = current + (x*more)
            y = nexty
    return matrix


def printmatindex(matrix):
    colmask = "%3d " * matrix.shape[1]
    for i in range(matrix.shape[0]):
        print colmask % tuple(matrix[i,:])

In [ ]:
#filedata, filerate = musictoys.audiofile.read("audio_files/kronfeld-dreamatic.wav")
#filedata, filerate = musictoys.audiofile.read("audio_files/jfb-back_home.wav")
#filedata, filerate = musictoys.audiofile.read("audio_files/liberty_chaps-get_up_get_down.wav")
#filedata, filerate = musictoys.audiofile.read("/home/mars/Music/Track Library/5175913_Black_Noir_Schwarz_feat__Renegade_Original_Mix.mp3")
filedata, filerate = musictoys.audiofile.read("/home/mars/Music/Track Library/7536385_Undulator_Original_Mix.mp3")
samples, samplerate = musictoys.analysis.normalize(filedata, filerate)

In [ ]:
pathmatrix = spacefillpath(256, 256)
print "number of samples == %d, number of steps in path == %d" % (len(samples), pathmatrix.size)
step_size = len(samples) / float(pathmatrix.size)
print "step_size == %f" % step_size

# how big an FFT will we need for this step size?
clip_size = int(2 ** ( np.ceil(np.log(step_size) / np.log(2)) ))
print "clip_size == %d" % clip_size

In [ ]:
energies = np.zeros(pathmatrix.size)
powers = np.zeros(pathmatrix.size)
centroids = np.zeros(pathmatrix.size)
rolloffs = np.zeros(pathmatrix.size)
crests = np.zeros(pathmatrix.size)
variances = np.zeros(pathmatrix.size)
entropies = np.zeros(pathmatrix.size)
flatnesses = np.zeros(pathmatrix.size)

spec_size = int(clip_size/2)
spectrumbucket = np.zeros(spec_size, dtype=np.float)
spec_count = 0

for i in np.arange(pathmatrix.size-1):
    start = int(np.floor(i * step_size))
    stop = min(start+clip_size, len(samples))
    clip = samples[start:stop]
    energies[i] = np.abs(clip).sum() / clip_size
    # compute rms
    powers[i] = np.sqrt(np.square(clip).sum()) / clip_size
    # get spectrum, compute centroid & crest
    spectrum = np.abs(np.fft.rfft(clip))[:spectrumbucket.shape[0]] / spec_size
    centroids[i] = musictoys.spectral.centroid(spectrum, samplerate)
    rolloffs[i] = musictoys.spectral.rolloff(spectrum, samplerate)
    crests[i] = musictoys.spectral.crest(spectrum)
    variances[i] = musictoys.spectral.variance(spectrum)
    entropies[i] = musictoys.spectral.entropy(spectrum)
    flatnesses[i] = musictoys.spectral.flatness(spectrum, 2.0)
    # compute some helpful sats which will hopefully help us understand this all better
    np.add(spectrum, spectrumbucket, out=spectrumbucket)
    spec_count += 1
    
spectrumbucket /= float(spec_count)
energies -= energies.min()
powers -= powers.min()
powers /= energies.max()
energies /= energies.max()

In [ ]:
fig, axarr = plot.plt.subplots(2, 3)
fig.set_figheight(6)

axarr[0,0].hist(centroids, bins=50)
axarr[0,0].set_title("Centroids")

axarr[0,1].hist(rolloffs, bins=50)
axarr[0,1].set_title("Rolloff freq")

axarr[0,2].hist(crests, bins=50)
axarr[0,2].set_title("Crest factor")

axarr[1,0].hist(variances, bins=50)
axarr[1,0].set_title("Variance")

axarr[1,1].hist(entropies, bins=50)
axarr[1,1].set_title("Entropy")

axarr[1,2].hist(flatnesses, bins=50)
axarr[1,2].set_title("Flatness")

plot.plt.show()

In [ ]:
plot.plt.figure()
plot.plt.imshow(energies[pathmatrix], aspect='equal', cmap='gray', interpolation='nearest')


In [ ]:
red_layer = np.square(1.0 - np.multiply(low_bias, energies)[pathmatrix])
green_layer = (1.0 - energies[pathmatrix])
blue_layer = np.square(1.0 - np.multiply(hi_bias, energies)[pathmatrix])
image_rgb = np.stack((red_layer, green_layer, blue_layer), axis=2)
image_rgb = image_rgb ** 1.5
plot.plt.figure(figsize=(8,8))
plot.plt.imshow(image_rgb, aspect='equal', interpolation='nearest')

In [ ]:
def hilbert_num_to_point(m, d):
    """
    m is the width/height of the hilbert square
    d is the hilbert number, which therefore must be in the range 0..m**2
    returns X,Y
    """

    n = 2 ** m

    x = 0
    y = 0
    t = d
    s = 1

    while ( s < n ):
        rx = ( ( t // 2 ) % 2 )
        if ( rx == 0 ):
          ry = ( t % 2 )
        else:
          ry = ( ( t ^ rx ) % 2 )
        x, y = rot ( s, x, y, rx, ry )
        x = x + s * rx
        y = y + s * ry
        t = ( t // 4 )

        s = s * 2

    return x, y

def rot ( n, x, y, rx, ry ):

#*****************************************************************************80
#
## ROT rotates and flips a quadrant appropriately.
#
#  Licensing:
#
#    This code is distributed under the GNU LGPL license. 
#
#  Modified:
#
#    03 January 2016
#
#  Parameters:
#
#    Input, integer N, the length of a side of the square.  
#    N must be a power of 2.
#
#    Input/output, integer X, Y, the coordinates of a point.
#
#    Input, integer RX, RY, ???
#
  if ( ry == 0 ):
#
#  Reflect.
#
    if ( rx == 1 ):
      x = n - 1 - x
      y = n - 1 - y
#
#  Flip.
#
    t = x
    x = y
    y = t

  return x, y

In [ ]:
# let's try the same project with a hilbert curve
hilbuf = np.zeros((pathmatrix.shape[0], pathmatrix.shape[0]))

for i in np.arange(128*128):
    x, y = hilbert_num_to_point(128, i)
    hilbuf[y,x] = powers[i]

In [ ]:
fig, axarr = plot.plt.subplots(1, 2)
fig.set_figheight(6)
fig.set_figwidth(12)

axarr[0].imshow(imagebuffer, aspect='equal', cmap='gray', interpolation='nearest')
axarr[0].set_title("Martian quadtree")

axarr[1].imshow(hilbuf, aspect='equal', cmap='gray', interpolation='nearest')
axarr[1].set_title("Hilbert curve")

plot.plt.show()